In [1]:
from kernel_est_funcs import *
import numpy as np

In [2]:
spikes = np.load("spikes-10e4-ms.npy")

In [21]:
tau = 100 #ms
#calcium_signal = sim_calcium(spikes, tau)
savgol_win_len =51

methods = ['pure_fit', 'iqr_outlier', 'ransac_outlier', 'zscore_outlier']
tau_sim = np.array([10, 20, 50, 100, 200, 300, 500, 800, 1000, 1500, 2000, 3000, 5000, 8000, 10000])
tau_est_pure_fit = np.zeros(tau_sim.size)
tau_est_iqr_outlier = np.zeros(tau_sim.size)
tau_est_ransac_outlier = np.zeros(tau_sim.size)
tau_est_zscore_outlier = np.zeros(tau_sim.size)
tau_est_iqr_ransac = np.zeros(tau_sim.size)

for names in methods:
    method = globals()[names]
    counter = 0
    for i in tau_sim:
        calcium_signal = sim_calcium(spikes, i)
        signal, deriv = smoothed_signals(calcium_signal, savgol_win_len)
        var_name = 'tau_est_'+names
        globals()[var_name][counter] = -1/method(signal, deriv)
        counter +=1



In [24]:
%matplotlib qt
fig, ax = plt.subplots()

ax.plot(tau_sim, tau_est_pure_fit, 'o-', label='with outliers')
ax.plot(tau_sim, tau_est_iqr_outlier, 'o-', label='IQR')
ax.plot(tau_sim, tau_est_ransac_outlier, 'o-', label= 'RANSAC')
ax.plot(tau_sim, tau_est_zscore_outlier, 'o-', label='Z score')

ax.set_xscale('log')
#ax.set_yscale('log')
ax.legend()